# Montecarlo Approximation of Electron-Matter-Interaction

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from fastai.vision.all import * 
import fastai
import torch
print("sys.version", sys.version)
print("cuda device name(0)", torch.cuda.get_device_name(0))
print("torch.__version__", torch.__version__)
print("fastai.__version__", fastai.__version__)

sys.version 3.9.15 (main, Nov 24 2022, 14:31:59) 
[GCC 11.2.0]
cuda device name(0) Tesla V100S-PCIE-32GB
torch.__version__ 1.12.1
fastai.__version__ 2.7.10


# Preprocessing

In [3]:
import pathlib
input_path = Path('./volumes/')
output_path = Path('./data/')

In [4]:
from PIL import Image
import numpy
import os

def get_tif_images(file_path):
    files = get_image_files( file_path )
    files =  [file for file in files if "_GT" in str(file)]
    return files

def load_multislice_tif_as_np_array(file_path, max_slices = -1):
    image = Image.open(file_path)
    h,w = numpy.shape( image )
    slices_to_load = image.n_frames
    if max_slices != -1 and max_slices < image.n_frames:
        slices_to_load = max_slices
    volume = numpy.empty( (w,h,slices_to_load), dtype=numpy.uint8 )
    for i in range(slices_to_load):
        image.seek(i)
        image_slice = numpy.array( image )
        if image_slice.dtype == numpy.float32:
            image_slice = image_slice * 255.
            image_slice = image_slice.astype( numpy.int8 )
        volume[:,:,i] = image_slice
    return volume; 

def image_list_to_np_array( image_list ):
    size_z = len(image_list)
    size_x,size_y = image_list[0].shape
    result = numpy.zeros( (size_x,size_y,size_z), dtype=numpy.uint8  )
    # result = numpy.expand_dims( result, 2 )
    for i,next_slice in enumerate(image_list):
        result[:,:,i] = numpy.array( next_slice )
    return result


def load_multislice_tif(file_path, max_slices = -1):
    image = Image.open(file_path)
    h,w = numpy.shape( image )
    slices_to_load = image.n_frames
    if max_slices != -1 and max_slices < image.n_frames:
        slices_to_load = max_slices
    imagelist = [] 
    for i in range(slices_to_load):
        image.seek(i)
        image_slice = numpy.array( image )
        if image_slice.dtype == numpy.float32:
            image_slice = image_slice * 255.
            image_slice = image_slice.astype( numpy.int8 )
        imagelist.append( PILImage.create( image_slice ) )
    return imagelist; 

def image_list_to_np_array( image_list ):
    size_z = len(image_list)
    size_x,size_y = image_list[0].shape
    result = numpy.zeros( (size_x,size_y,size_z), dtype=numpy.uint8  )
    # result = numpy.expand_dims( result, 2 )
    for i,next_slice in enumerate(image_list):
        result[:,:,i] = numpy.array( next_slice )
    return result

def create_extended_heightfield_from_volume( volume_data, n_layers, exit = False ):
    volume_data = volume_data.copy()
    
    _,_,depth = volume_data.shape
    
    # the zero will be rolled to the top layer, which makes sure that the first layer is reported as first hit
    # if the surfaces reaches the top of the volume
    volume_data[:,:,depth-1] = 0
    shifted_volume = numpy.roll( volume_data, shift=1, axis=2)
    # this fixes the problem that no maximum exists if you can look though the volume
    volume_data[:,:,depth-1] = 1
    
    if not exit:
        mask = (volume_data == 1) & (shifted_volume == 0)
    else:
        mask = (volume_data == 0) & (shifted_volume == 1)
    
    x,y,z = np.where( mask )             
    size_x,size_y,_ = volume_data.shape
    result = []
    
    tmp_data = numpy.empty((size_x,size_y), dtype=object)
    for i in range(size_x):
        for j in range(size_y):
            tmp_data[i,j] = []
            
    for i in range(len(x)):
        tmp_data[x[i],y[i]].append(z[i])
        
    for i in range(n_layers):    
        height_field = numpy.zeros( ( size_x, size_y ), dtype=numpy.uint16 )
        for xx in range(size_x):
            for yy in range(size_y):
                if i < len(tmp_data[xx,yy]):
                    height_field[xx,yy] = tmp_data[xx,yy][i]                    
                else:
                    height_field[xx,yy] = depth       
                
        height_field = Image.fromarray( height_field, 'I;16' )
        result.append(height_field)
        
    return result

In [5]:
def get_base_stack_name( filename ):
    filename_parts = str(filename.stem).split("_")
    y = Path( "_".join(filename_parts[:-1]) + "_" + filename_parts[-1] )
    return str(y)

def get_gt_stack_name( filename ):
    filename_parts = str(filename.stem).split("_")
    y = Path( "_".join(filename_parts[:-1]) + "_GT" + ".tif" )
    return y

def get_bse_stack_name( filename ):
    filename_parts = str(filename.stem).split("_")
    y = Path( "_".join(filename_parts[:-1]) + "_BSE" + ".tif" )
    return y

In [6]:
def compute_slice(depth,slice_depth, visible_depth_of_slab):
    # this optimization prevents a lot of overdraw for volumes with high density
    if not visible_depth_of_slab is None:
        if depth > visible_depth_of_slab:
            depth = visible_depth_of_slab            
    result = []
    for layer in range( int( (depth-1)/slice_depth )+1 ):
        ifrom = layer*slice_depth
        ito   = (layer+1)*slice_depth
        if ito > depth:
            ito=depth
        result.append((ifrom,ito))
    return result

def compute_mask(resolution,ifrom,ito):
    mask = numpy.full(resolution,False)
    mask[:,:,ifrom:ito] = True
    return mask

In [7]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'
from skimage import measure
from scipy.ndimage import gaussian_filter
import trimesh as tri
import pyrender
import gc

import multiprocessing as mp

def create_mesh_from_slab( volume_data, ifrom, ito ):
    print("  extracting iso")
    volume_resolution = volume_data.shape    
    mask = compute_mask( volume_data.shape, ifrom, ito )
    try:
        verts, faces, normals, values = measure.marching_cubes( volume_data, method="lorensen", mask=mask )    
        verts -= 1.0 # correct shift that results from padding
        verts /= (volume_resolution[0]-3) # -1 ensures vertices of mesh finally lie at corners of pixels

        normals[:,2] *= -1 # invert normals.z so normal map looks blue as convention dictates
        normals += 1.0
        normals /= 2.0
        vertex_colors = tri.visual.color.ColorVisuals(vertex_colors=normals)
        print("  creating mesh")
        surface = tri.base.Trimesh(vertices=verts,faces=faces, visual=vertex_colors)    

        return pyrender.Mesh.from_trimesh( surface, smooth=True )
    
    except RuntimeError:
        return None

def create_mesh_from_volume( volume_data, visible_depth_of_slab ):
    print( "extracting surface from volume (marching cubes)" )
    
    use_cpu = mp.cpu_count() - 1
    if use_cpu > 8:
        use_cpu = 8 # avoids out of memory

    while use_cpu >= 1:
        try:       
            pool = mp.Pool(use_cpu)

            # pad data with zero so you cannot look into objects
            volume_data = numpy.pad( volume_data, pad_width = ((1,1),(1,1),(1,1)), mode='constant', constant_values=0.0 )
            # volume_data = gaussian_filter(volume_data, sigma=0.25)

            slabs = reversed( compute_slice( volume_data.shape[2], 16, visible_depth_of_slab ) )

            meshes = pool.starmap( create_mesh_from_slab, [(volume_data,ifrom,ito) for ifrom,ito in slabs] )
            return meshes
        except MemoryError:
            use_cpu = int( use_cpu / 2 )
            print("memory allocation failed, reducing multithreading to", use_cpu)

def render_meshes( meshes, resolution ):
    # print("setting up scene")
    scene = pyrender.Scene( bg_color=[0, 0, 0] )
    camera = pyrender.OrthographicCamera(xmag=1.0, ymag=1.0)
    
    for mesh in meshes:
        if mesh is None:
            continue
        scene.add(mesh, pose=np.eye(4))

    # quaternion: rotate 90 degree around z direction
    camera_node = pyrender.Node(camera=camera, rotation=( 0, 0, 0.7071068, 0.7071068 ), translation=(0.5, 0.5, -10), scale=(0.5, 0.5, -0.5))
    scene.add_node(camera_node)

    # render scene
    r = pyrender.OffscreenRenderer( resolution[0]*4, resolution[1]*4 )
    flags = pyrender.constants.RenderFlags.FLAT | pyrender.constants.RenderFlags.ALL_SOLID | pyrender.constants.RenderFlags.SKIP_CULL_FACES
    print("rendering")
    color, depth = r.render(scene, flags=flags)
    r.delete()
    
    return color

def create_normalmap_from_volume( volume_data, resolution, visible_depth_of_slab ):
    print("creating normal map of ", resolution)
    meshes = create_mesh_from_volume( volume_data, visible_depth_of_slab )
    normal_map = render_meshes( meshes, resolution )
    normal_map = Image.fromarray( normal_map )   
    normal_map.thumbnail((resolution[0],resolution[1]), Image.ANTIALIAS)
    return normal_map

In [11]:
from os.path import exists

do_overwrite   = True
do_heightfield = False
do_normal      = False
do_bse         = False
do_se          = False
do_ground_truth= False
do_cleanup     = True

def create_heightfields( gt_volume, stackname, top, do_overwrite ):
    n_layer = 2

    output_hf_filenames = []
    for layer in range(n_layer):              
        output_hf_filenames.append( output_path / Path( stackname + "_" + str(top) + "_entry_hf_" + str(layer) + ".tif" ) )
        output_hf_filenames.append( output_path / Path( stackname + "_" + str(top) + "_exit_hf_" + str(layer) + ".tif" ) )
        
    exist_all = True
    for x_hf_filename in output_hf_filenames:
        if not exists(x_hf_filename):
            exist_all = False

    if exist_all and not do_overwrite:
        print("all heightfields exists, skipping", stackname )
        return None
                                   
    print("creating heightfield", stackname)
    x_hf = create_extended_heightfield_from_volume( gt_volume, n_layer, exit=False )                                    
    visible_depth_of_slab = numpy.amax(x_hf[0])
    print("  slab has maximum visible depth of ", visible_depth_of_slab )
    for layer in range(n_layer):            
        x_hf_filename = output_path / Path( stackname + "_" + str(top) + "_entry_hf_" + str(layer) +".tif" )   
        print("  saving x heightfield to ", x_hf_filename)
        x_hf[layer].save( x_hf_filename )
    x_hf = create_extended_heightfield_from_volume( gt_volume, n_layer, exit=True ) 
    for layer in range(n_layer):            
        x_hf_filename = output_path / Path( stackname + "_" + str(top) + "_exit_hf_" + str(layer) +".tif" )   
        print("  saving x heightfield to ", x_hf_filename)
        x_hf[layer].save( x_hf_filename )
    return visible_depth_of_slab    

for gt_stack_name in os.listdir(input_path):
    # bse_stack_name = "Cylinder_Vv03_r10-20_h300-500_Num2_BSE.tif"
    print("checking", gt_stack_name)
    if "_GT" not in gt_stack_name:
        continue
           
    bse_stack_name = get_bse_stack_name( Path( gt_stack_name ) )
    print(gt_stack_name, "/", bse_stack_name)

    if do_bse:       
        bse_stack = load_multislice_tif( input_path / bse_stack_name )
        print("bse_stack", bse_stack_name, len(bse_stack) )
        stack_length = len(bse_stack)

    if do_se:       
        se_stack_name = bse_stack_name.replace('_BSE', '_SE')
        se_stack = load_multislice_tif( input_path / se_stack_name )
        print("se_stack", se_stack_name, len(se_stack) )
        stack_length = len(se_stack)
    
    if do_heightfield or do_ground_truth or do_cleanup:                        
        print("loading gt_stack", gt_stack_name )
        gt_stack = load_multislice_tif_as_np_array( input_path / gt_stack_name )
        print("gt_stack", gt_stack_name, gt_stack.shape )
        stack_length = len(gt_stack)
           
    stackname = get_base_stack_name( Path( bse_stack_name ) )

    slice_thickness = 10
    use_percentage = 0.25
    over_length = stack_length % slice_thickness
    stack_length = stack_length-over_length
    top_of_slab = int( stack_length*use_percentage )
    
    for top in reversed( range( 0, top_of_slab, slice_thickness) ):        
        visible_depth_of_slab = None
        
        if do_heightfield:
            gt_volume = gt_stack[:,:,top:-1]
            resolution = gt_volume.shape
            visible_depth_of_slab = create_heightfields( gt_volume, stackname, top, do_overwrite );

        if do_normal:   
            x_n_filename = output_path / Path( stackname + "_" + str(top) + "_normal.tif" )
            if not do_overwrite:
                if exists(x_n_filename):
                    print("normal map exists, skipping",x_n_filename)
                    continue
            x_n  = create_normalmap_from_volume( gt_volume, resolution, visible_depth_of_slab ) 
            print("saving x normal map to ", x_n_filename)
            x_n.save( x_n_filename )

        if do_bse:   
            bse_filename = output_path / Path( stackname + "_" + str(top) + "_bse.tif" )
            if not do_overwrite:
                if exists(bse_filename):
                    print("skipping",bse_filename)
                    continue
            y = bse_stack[top]            
            print("saving bse to ", bse_filename)
            y.save( bse_filename )
            
        if do_se:   
            se_filename = output_path / Path( stackname + "_" + str(top) + "_se.tif" )
            if not do_overwrite:
                if exists(se_filename):
                    print("skipping",se_filename)
                    continue
            y = se_stack[top]
            print("saving se to", se_filename)
            y.save( se_filename )
            
        if do_ground_truth:
            gt_filename = output_path / Path( stackname + "_" + str(top) + "_gt.tif" )
            if not do_overwrite:
                if exists(gt_filename):
                    print("skipping",gt_filename)
                    continue
            y = PILImage.create( gt_stack[:,:,top] )
            print("saving gt to", gt_filename)
            y.save( gt_filename )
            
    if do_cleanup:
        print("checking for cleanup")
        last_top = int( ( top_of_slab / slice_thickness ) + 1 ) * slice_thickness
        for top in range( last_top, stack_length, slice_thickness):
            bse_filename = output_path / Path( stackname + "_" + str(top) + "_bse.tif" )
            se_filename = output_path / Path( stackname + "_" + str(top) + "_se.tif" )
            gt_filename = output_path / Path( stackname + "_" + str(top) + "_gt.tif" )
            print("  checking",bse_filename)
            if os.path.isfile(bse_filename):
                print("  found excess file",bse_filename,"removing")
                os.remove(bse_filename)
            print("  checking",se_filename)
            if os.path.isfile(se_filename):
                print("  found excess file",se_filename,"removing")
                os.remove(se_filename)
            print("  checking",gt_filename)
            if os.path.isfile(gt_filename):
                print("  found excess file",gt_filename,"removing")
                os.remove(gt_filename)

checking Cylinder_Vv05_r5-10_h100-150_Num2_BSE.tif
checking Cylinder_Vv03_r5-10_h100-150_homogen_GT.tif
Cylinder_Vv03_r5-10_h100-150_homogen_GT.tif / Cylinder_Vv03_r5-10_h100-150_homogen_BSE.tif
loading gt_stack Cylinder_Vv03_r5-10_h100-150_homogen_GT.tif
gt_stack Cylinder_Vv03_r5-10_h100-150_homogen_GT.tif (850, 850, 1399)
checking for cleanup
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_220_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_220_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_220_gt.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_230_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_230_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_230_gt.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_240_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_240_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_homogen_BSE_240_gt.tif
  checking data/Cylinder_Vv03_r5-10_h1

gt_stack Cylinder_Vv03_r5-10_h100-150_Num2_GT.tif (850, 850, 1399)
checking for cleanup
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_220_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_220_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_220_gt.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_230_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_230_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_230_gt.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_240_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_240_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_240_gt.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_250_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_250_se.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_250_gt.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_260_bse.tif
  checking data/Cylinder_Vv03_r5-10_h100-150_Num2_BSE_260_se.t

gt_stack Sphere_Vv05_r30-50_Num2_ITWMconfig_GT.tif (850, 850, 1101)
checking for cleanup
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_220_bse.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_220_se.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_220_gt.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_230_bse.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_230_se.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_230_gt.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_240_bse.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_240_se.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_240_gt.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_250_bse.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_250_se.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_250_gt.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconfig_BSE_260_bse.tif
  checking data/Sphere_Vv05_r30-50_Num2_ITWMconf

KeyboardInterrupt: 

##  Double Check

In [ ]:
hf = []

max_depth = 0

for layer in ["0", "1"]:
    hf_entry = Image.open( output_path / Path( "Cylinder_Vv07_r5-10_h100-150_homogen_BSE_20_entry_hf_" + layer + ".tif" ) )
    hf_exit =  Image.open( output_path / Path( "Cylinder_Vv07_r5-10_h100-150_homogen_BSE_20_exit_hf_" + layer  + ".tif" ) )
    hf_entry = numpy.array(hf_entry)
    hf_exit = numpy.array(hf_exit)
    if numpy.amax(hf_entry) > max_depth:
        max_depth = numpy.amax(hf_entry)
    if numpy.amax(hf_exit) > max_depth:
        max_depth = numpy.amax(hf_exit)
    hf.append( ( hf_entry,hf_exit ) )

entry,exit = hf[0]
result_shape = (entry.shape[0], entry.shape[1], max_depth)
    
result = numpy.zeros(result_shape, dtype=np.uint8)

for layer in [0, 1]:
    entry,exit = hf[layer]
    for y in range(entry.shape[1]):
        for x in range(entry.shape[0]):
            entry_value = entry[x,y]
            exit_value  = exit[x,y]
            result[x,y,entry_value:exit_value] = 1
            
image_list = []
for volume_slice in result:
    image_list.append(Image.fromarray(volume_slice))

image_list[0].save("extended_heightfield_test.tif", compression="tiff_deflate", save_all=True, append_images=image_list[1:])